In [ ]:
%matplotlib notebook
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import os
import pandas as pd
import sklearn.cluster
import itertools
from IPython.display import display
from experimental.pokerbots import hand_evaluator_python as hep, bin_centers_pb2


In [ ]:
data_directory = os.path.expanduser('~/code/robot/experimental/pokerbots/cluster_hands')

In [ ]:
os.listdir(data_directory)

In [ ]:
hole_card_data = pd.read_csv(os.path.join(data_directory, 'hole_cards_output.csv'))

In [ ]:
def plot_cdf(data):
    sorted_data = sorted(data)
    frac = np.linspace(0, 1.0, len(sorted_data))
    plt.plot(sorted_data, frac)

In [ ]:
plt.figure(figsize=(15, 5))
plt.subplot(131)
plot_cdf(hole_card_data['strength'])
plt.ylabel('Fraction')
plt.title('Strength')
plt.subplot(132)
plot_cdf(hole_card_data['pos_pot'])
plt.title('Positive Potential')
plt.subplot(133)
plot_cdf(hole_card_data['neg_pot'])
plt.title('Negative Potential')
plt.tight_layout()

In [ ]:
k_means_fit = sklearn.cluster.KMeans(n_clusters=12, n_init=20)
hole_card_data['cluster_idx'] = k_means_fit.fit_predict(hole_card_data[['strength', 'neg_pot', 'pos_pot']])

In [ ]:
ax = plt.figure().add_subplot(projection='3d')
for (i, group) in hole_card_data.groupby('cluster_idx'):
    ax.plot(group.neg_pot, group.pos_pot, group.strength, 'o', label=i)
plt.legend()
ax.set_xlabel('Negative Potential')
ax.set_ylabel('Postive Potential')
ax.set_zlabel('Strength')
plt.tight_layout()

In [ ]:
cmap(hole_card_data['cluster_idx']).T.shape

In [ ]:
for (i, group) in hole_card_data.groupby('cluster_idx'):
    print('Cluster:', i)
    print(group.mean(numeric_only=True))
    print(list(group['card1'] + group['card2']))

In [ ]:
flop_directory = os.path.join(data_directory, 'flop_cards')

flop_cards = []
for filename in os.listdir(flop_directory):
    flop_cards.append(pd.read_csv(os.path.join(flop_directory, filename)))
    flop_cards[-1]['card_1'] = filename[:2]
    flop_cards[-1]['card_2'] = filename[2:4]
flop_cards = pd.concat(flop_cards)

In [ ]:
flop_k_means = sklearn.cluster.KMeans(n_clusters=100, n_init=10)
flop_cards['cluster_idx'] = flop_k_means.fit_predict(flop_cards[['strength', 'neg_pot', 'pos_pot']])

In [ ]:
for (i, group) in flop_cards.groupby('cluster_idx'):
    print('Cluster:', i, 'length:', len(group))
    print(group.mean(numeric_only=True))
    display(group.sample(20))

In [ ]:
flop_cards.to_pickle(os.path.join(flop_directory, 'flop_data.p'))

In [ ]:
samples = []
SAMPLES_PER_GROUP = 5000
for (i, group) in flop_cards.groupby('cluster_idx'):
    sample = group.sample(5000)
    hands = list(sample['card_1'] + sample['card_2'] + sample['board1'] + sample['board2'] + sample['board3'])
    samples.append(hands)
samples = list(itertools.chain(*samples))


In [ ]:
deck = [''.join(x) for x in itertools.product('23456789TJQKA', 'shcd')]

In [ ]:
to_evaluate = []
for hand in samples:
    for card in deck:
        if card in hand:
            continue
        to_evaluate.append(hand + card)

In [ ]:
turn_results = hep.evaluate_hands(to_evaluate, 5, 1000)

In [ ]:
turn_data = []
for hand, result in zip(to_evaluate, turn_results):
    turn_data.append([hand[2*i:2*i+2] for i in range(len(hand) // 2)] + [result.strength, result.negative_potential, result.positive_potential])
turn_data = pd.DataFrame(turn_data, columns=['card1', 'card2', 'board1', 'board2', 'board3', 'board4', 'strength', 'neg_pot', 'pos_pot'])

In [ ]:
turn_data

In [ ]:
turn_k_means = sklearn.cluster.KMeans(n_clusters=100, n_init=10)
turn_data['cluster_idx'] = turn_k_means.fit_predict(turn_data[['strength', 'neg_pot', 'pos_pot']])

In [ ]:
for (i, group) in turn_data.groupby('cluster_idx'):
    print('Cluster:', i, 'length:', len(group))
    print(group.mean(numeric_only=True))
    display(group.sample(20))

In [ ]:
turn_data.to_pickle(os.path.join(data_directory, 'turn_data.p'))

In [ ]:
samples = []
SAMPLES_PER_GROUP = 5000
for (i, group) in turn_data.groupby('cluster_idx'):
    sample = group.sample(5000)
    hands = list(sample['card1'] + sample['card2'] + sample['board1'] + sample['board2'] + sample['board3'] + sample['board4'])
    samples.append(hands)
samples = list(itertools.chain(*samples))

In [ ]:
deck = [''.join(x) for x in itertools.product('23456789TJQKA', 'sc')]

In [ ]:
to_evaluate = []
for hand in samples:
    for card in deck:
        if card in hand:
            continue
        to_evaluate.append(hand + card)

In [ ]:
len(to_evaluate)

In [ ]:
river_results = hep.evaluate_hands(to_evaluate, 5, 1000)

In [ ]:
river_data = []
for hand, result in zip(to_evaluate, river_results):
    river_data.append([hand[2*i:2*i+2] for i in range(len(hand) // 2)] + [result.strength, result.negative_potential, result.positive_potential])
river_data = pd.DataFrame(river_data, columns=['card1', 'card2', 'board1', 'board2', 'board3', 'board4', 'board5', 'strength', 'neg_pot', 'pos_pot'])

In [ ]:
river_k_means = sklearn.cluster.KMeans(n_clusters=100, n_init=10)
river_data['cluster_idx'] = river_k_means.fit_predict(river_data[['strength', 'neg_pot', 'pos_pot']])

In [ ]:
for (i, group) in river_data.groupby('cluster_idx'):
    print('Cluster:', i, 'length:', len(group))
    print(group.mean(numeric_only=True))
    display(group.sample(20))

In [ ]:
river_data.to_pickle(os.path.join(data_directory, 'river_data.p'))

In [ ]:
for (name, df) in [['preflop', hole_card_data], ['flop', flop_cards],['turn', turn_data],['river',river_data]]:
    buckets = df.groupby('cluster_idx').apply(lambda x: x.mean(numeric_only=True))
    buckets.to_csv(os.path.join(data_directory, name + '_buckets.csv'))

In [ ]:
file_names = ['preflop_buckets.csv', 'flop_buckets.csv', 'river_buckets.csv', 'turn_buckets.csv']
dfs = {}
for fname in file_names:
    dfs[fname.split('_')[0]] = pd.read_csv(os.path.join(data_directory, fname))

In [ ]:
bin_centers = bin_centers_pb2.PerTurnBinCenters()

for i, row in dfs['preflop'].iterrows():
    bin_center = bin_centers_pb2.BinCenter()
    bin_center.strength = row.strength
    bin_center.negative_potential = row.neg_pot
    bin_center.positive_potential = row.pos_pot
    bin_centers.preflop_centers.append(bin_center)

for i, row in dfs['flop'].iterrows():
    bin_center = bin_centers_pb2.BinCenter()
    bin_center.strength = row.strength
    bin_center.negative_potential = row.neg_pot
    bin_center.positive_potential = row.pos_pot
    bin_centers.flop_centers.append(bin_center)
    
for i, row in dfs['turn'].iterrows():
    bin_center = bin_centers_pb2.BinCenter()
    bin_center.strength = row.strength
    bin_center.negative_potential = row.neg_pot
    bin_center.positive_potential = row.pos_pot
    bin_centers.turn_centers.append(bin_center)
    
for i, row in dfs['river'].iterrows():
    bin_center = bin_centers_pb2.BinCenter()
    bin_center.strength = row.strength
    bin_center.negative_potential = row.neg_pot
    bin_center.positive_potential = row.pos_pot
    bin_centers.river_centers.append(bin_center)

In [ ]:
with open('/home/erick/code/robot/experimental/pokerbots/bin_centers.pb', 'wb') as file_out:
    file_out.write(bin_centers.SerializeToString())